In [ ]:
from tabula import read_pdf
import re
from tika import parser
import pandas as pd
import numpy as np
import pprint

In [ ]:
academic_history = read_pdf("",
                            pages = 'all',
                            multiple_tables = True,
                            lattice=True) #path to transcript file

In [ ]:
raw = parser.from_file('')#path to audit file
degree_audit_string = raw['content']

degree_audit_string_major = degree_audit_string[
    degree_audit_string.find('Lower Division Requirements'):
    degree_audit_string.find('48 Upper Division Unit Requirement')
]

degree_audit_string_major_and_minor = degree_audit_string_major + degree_audit_string[
    degree_audit_string.find('MINOR REQUIREMENTS'):
    degree_audit_string.find('WORK IN PROGRESS')
]
degree_audit_string_major_and_minor

In [ ]:
academic_history[4]

In [ ]:
list_of_correct_indicies = []

for i in np.arange(len(academic_history)):
        
    if academic_history[i][0][0] == 'Subject':
        list_of_correct_indicies.append(i)

In [ ]:
list_of_correct_indicies

In [ ]:
final_list = []

for item in np.arange(len(list_of_correct_indicies)):
    
    final_list.append(academic_history[list_of_correct_indicies[item]])

In [ ]:
print(final_list)

In [ ]:
for i in np.arange(len(final_list)):
    final_list[i].columns = final_list[i].iloc[0]
    final_list[i] = final_list[i].iloc[1:]

In [ ]:
final_list[1]

In [ ]:
for item in final_list:
    item["Course Code"] = item["Subject"].map(str) + " " + item["Course"]

In [ ]:
final_list[0]

In [ ]:
list_of_courses_done = []

for obj in final_list:
    for item in (obj['Course Code']):
        list_of_courses_done.append(item)

for i in np.arange(len(final_list[-1]['Subject'])):
    list_of_courses_done.append(final_list[-1]['UCSD\rApprox'][i+1])

#This line removes the nan elements
list_of_courses_done = [x for x in list_of_courses_done if x == x]

for i in np.arange(len(list_of_courses_done)):
    list_of_courses_done[i] = list_of_courses_done[i].split('\r')

list_of_courses_done

In [ ]:
flat_list = []
for sublist in list_of_courses_done:
    for item in sublist:
        flat_list.append(item)

#Overriding list_of_courses_done
list_of_courses_done = flat_list
list_of_courses_done

In [ ]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [ ]:
courses_titles = 'MATH 180B,180C,189 COGS 102A,102B,102C,109,110,115,118A,118B,118C,118D,119'

def course_title(string_course_names):
        
    final_courses = []
    counter = ''
        
    for item in string_course_names.split():
        item = (item.split(','))
        
        if item[0].isalpha() and len(item[0]) == 3 or len(item[0]) == 4:
            counter = item[0]
        
        elif item[0].isalnum() and len(item[0]) < 5:
            for obj in item:
                final_courses.append(counter + " " + obj)
    
    return final_courses

In [ ]:
degree_audit_major_courses = course_title(degree_audit_string_major)
degree_audit_major_and_minor_courses = course_title(degree_audit_string_major_and_minor)
print(degree_audit_major_and_minor_courses)

In [ ]:
common_major_courses = list(set(list_of_courses_done).intersection(degree_audit_major_courses))
common_major_and_minor_courses = list(set(list_of_courses_done).intersection(degree_audit_major_and_minor_courses))
common_major_and_minor_courses

In [ ]:
total_major_points = 0.0
total_major_units = 0.0

for item in final_list[0:-1]:
    for i in np.arange(len(item['Course Code'])):
        if item['Course Code'][i+1] in common_major_courses:
            if type(item['Grade'][i+1]) == str:
                
                total_major_points += float(item['Points'][i+1])
                total_major_units += float(item['Units'][i+1])

total_major_and_minor_points = 0.0
total_major_and_minor_units = 0.0

for item in final_list[0:-1]:
    for i in np.arange(len(item['Course Code'])):
        if item['Course Code'][i+1] in common_major_and_minor_courses:
            if type(item['Grade'][i+1]) == str:
                
                total_major_and_minor_points += float(item['Points'][i+1])
                total_major_and_minor_units += float(item['Units'][i+1])


In [ ]:
major_gpa = total_major_points/total_major_units
major_and_minor_gpa = total_major_and_minor_points/total_major_and_minor_units
print("Major GPA is: " + str(major_gpa))
print("Major and Minor GPA is: " + str(major_and_minor_gpa))